In [92]:
%pip install PyPDF2 

Note: you may need to restart the kernel to use updated packages.


In [95]:
from functools import reduce
import requests
import io
import os 
from time import sleep
import random 

import pandas as pd
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader

In [109]:
courses = "https://faculty.ksu.edu.sa/wesam/courses"

def get_all_hrefs_from_courses_page(url):
    read = requests.get(url)
    html_content = read.content
    soup = BeautifulSoup (html_content,"html.parser")
    return list("https://faculty.ksu.edu.sa" + link.get("href") for link in soup.find_all("a", href=True) if "course" in link.get("href") or "page" in link.get("href"))


def find_table_hyperlinks(url):
#     url = "https://faculty.ksu.edu.sa/wesam/course/198603"
    try:
        read = requests.get(url)
        html_content = read.content
        soup = BeautifulSoup (html_content,"html.parser")
        table = soup.find('table')
    except:
        sleep(random.random() * 4)
        return find_table_hyperlinks(url)
    
    links = []
    if table is None:
        return []
    for tr in table.findAll("tr"):
        trs = tr.findAll("td")
        for each in trs:
            try:
                link = each.find('a')['href']
                links.append(link)
            except:
                pass
    return (soup.find(attrs={"class": "node-title"}).text, links)


def download_file(url, local_prefex=""):
    try:
        sleep(random.random() * random.random() * random.random() * 7)
        local_filename = local_prefex + url.split('/')[-1]
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): 
                    f.write(chunk)
    except:
        download_file(url, local_prefex)

In [110]:
files = []
for hyperlink in hyperlinks:    
    files.append(find_table_hyperlinks(hyperlink))

In [ ]:
try:
        os.mkdir("data")
except:
        pass
for file_list in reversed(files):
    if not file_list:
        continue
    name, urls = file_list
    try:
        os.mkdir(f"data/{name}")
    except:
        pass
    for url in urls:
        download_file(url, f"data/{name}/")